In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,root_mean_squared_error,r2_score,mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:
da=pd.read_csv('Delivery_amazon.csv')

In [3]:
da.head()

,Unnamed: 0,Order_ID,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category,distance
0,0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120,Clothing,2.288918
1,1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Stormy,Jam,scooter,Metropolitian,165,Electronics,14.777225
2,2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130,Sports,1.136966
3,3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,105,Cosmetics,5.698649
4,4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,Metropolitian,150,Toys,4.510871


In [4]:
da=da.loc[:,~da.columns.str.contains('^Unnamed')]

In [5]:
da.columns

Index(['Order_ID', 'Agent_Age', 'Agent_Rating', 'Store_Latitude',
       'Store_Longitude', 'Drop_Latitude', 'Drop_Longitude', 'Order_Date',
       'Order_Time', 'Pickup_Time', 'Weather', 'Traffic', 'Vehicle', 'Area',
       'Delivery_Time', 'Category', 'distance'],
      dtype='object')

In [6]:
da['Order_Date'] = pd.to_datetime(da['Order_Date'])  # Convert to datetime if it's not already

# Extracting
da['Year'] = da['Order_Date'].dt.year
da['Month'] = da['Order_Date'].dt.month
da['DayOfWeek'] = da['Order_Date'].dt.dayofweek

In [7]:
da=da.drop(['Order_Date',
       'Order_Time', 'Pickup_Time'],axis=1)

In [8]:
da.head()

,Order_ID,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Weather,Traffic,Vehicle,Area,Delivery_Time,Category,distance,Year,Month,DayOfWeek
0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,Sunny,High,motorcycle,Urban,120,Clothing,2.288918,2022,3,5
1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,Stormy,Jam,scooter,Metropolitian,165,Electronics,14.777225,2022,3,4
2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,Sandstorms,Low,motorcycle,Urban,130,Sports,1.136966,2022,3,5
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,Sunny,Medium,motorcycle,Metropolitian,105,Cosmetics,5.698649,2022,4,1
4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,Cloudy,High,scooter,Metropolitian,150,Toys,4.510871,2022,3,5


In [9]:
da_num=da.select_dtypes(include=np.number)

In [10]:
da_num.head()

,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Delivery_Time,distance,Year,Month,DayOfWeek
0,37,4.9,22.745049,75.892471,22.765049,75.912471,120,2.288918,2022,3,5
1,34,4.5,12.913041,77.683237,13.043041,77.813237,165,14.777225,2022,3,4
2,23,4.4,12.914264,77.678400,12.924264,77.688400,130,1.136966,2022,3,5
3,38,4.7,11.003669,76.976494,11.053669,77.026494,105,5.698649,2022,4,1
4,32,4.6,12.972793,80.249982,13.012793,80.289982,150,4.510871,2022,3,5


In [11]:
da_num.dropna(inplace=True)
da_num

,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Delivery_Time,distance,Year,Month,DayOfWeek
0,37,4.9,22.745049,75.892471,22.765049,75.912471,120,2.288918,2022,3,5
1,34,4.5,12.913041,77.683237,13.043041,77.813237,165,14.777225,2022,3,4
2,23,4.4,12.914264,77.678400,12.924264,77.688400,130,1.136966,2022,3,5
3,38,4.7,11.003669,76.976494,11.053669,77.026494,105,5.698649,2022,4,1
4,32,4.6,12.972793,80.249982,13.012793,80.289982,150,4.510871,2022,3,5
...,...,...,...,...,...,...,...,...,...,...,...
43589,30,4.8,26.902328,75.794257,26.912328,75.804257,160,1.144920,2022,3,3
43590,21,4.6,0.000000,0.000000,0.070000,0.070000,180,11.007735,2022,2,2
43591,30,4.9,13.022394,80.242439,13.052394,80.272439,80,3.383273,2022,3,4
43592,20,4.7,11.001753,76.986241,11.041753,77.026241,130,4.558840,2022,3,0


In [12]:
X=da_num.drop(columns=['Delivery_Time'])
y=da_num['Delivery_Time']

In [13]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [14]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [15]:
gb_model=GradientBoostingRegressor()
gb_model.fit(X_train,y_train)

GradientBoostingRegressor()

In [16]:
y_pred=gb_model.predict(X_test)

In [17]:
rmse=root_mean_squared_error(y_test,y_pred)
rmse

43.45345129885774

In [18]:
mae=mean_absolute_error(y_test,y_pred)
mae

32.76603987236461

In [19]:
r2=r2_score(y_test,y_pred)
r2

0.2820034417202012

In [21]:
import mlflow
import mlflow.sklearn
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Set up MLflow experiment and tracking
mlflow.set_experiment("Gradient_Boosting")
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

# Generate predictions
y_pred = gb_model.predict(X_test)

# Compute evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5  # Root Mean Squared Error

# Start an MLflow run
with mlflow.start_run():
    mlflow.set_tag("Run name","gradient_Boosting")
    
    # Model parameters
    params = {
        "n_estimators": gb_model.get_params()["n_estimators"],  
        "learning_rate": gb_model.get_params()["learning_rate"]
    }

    
    # Log parameters
    mlflow.log_params(params)
    
    # Log evaluation metrics
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("MSE", mse)
    mlflow.log_metric("RMSE", rmse)

    # Log the trained model
    mlflow.sklearn.log_model(gb_model, "Gradient_Booster_Regressor")

print(" Successfully logged the Gradient Booster Regressor model in MLflow.")


2025/02/13 17:05:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run gaudy-mouse-359 at: http://127.0.0.1:5000/#/experiments/788480028050572998/runs/c25e6fbf56784c188df7be1a17ff893e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/788480028050572998
 Successfully logged the Gradient Booster Regressor model in MLflow.


In [23]:
model_name = "GradientBoosterReg"
run_id = "34f8417c92ff42abb4d3415b7e978eba"

model_uri = f"runs:/{run_id}/random_forest"

model_version = mlflow.register_model(model_uri=model_uri, name=model_name)

print(f"Model '{model_name}' successfully registered as version {model_version.version}.")


Successfully registered model 'GradientBoosterReg'.
2025/02/13 17:05:54 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: GradientBoosterReg, version 1


Model 'GradientBoosterReg' successfully registered as version 1.


Created version '1' of model 'GradientBoosterReg'.
